# TPC Analisador Léxico

### Enunciado:
##### Construir um analisador léxico para uma linguagem de query com a qual se podem escrever frases do género:



In [ ]:
"""
# DBPedia: obras de Chuck Berry
select ?nome ?desc where {
    ?s a dbo:MusicalArtist.
    ?s foaf:name "Chuck Berry"@en .
    ?w dbo:artist ?s.
    ?w foaf:name ?nome.
    ?w dbo:abstract ?desc
} LIMIT 1000 
"""

In [12]:
tokens = [
        ('NEWLINE',r"\n"),
        ('COMMENT', r"#.*"),
        ('STRING', r'"[^"]*"'),
        ('SELECT', r"\bselect\b"),
        ('WHERE', r"\bwhere\b"),
        ('LIMIT', r"\bLIMIT\b"),
        ('VAR', r"\?[A-Za-z_][A-Za-z0-9]*"),
        ('PREFIX', r"[A-Za-z_][A-Za-z0-9_]*:[A-Za-z_][A-Za-z0-9_]*"),
        ('IDIOMA', r"@[a-zA-Z\-]+"),
        ('A', r"\ba\b"),
        ('LBRACE', r"\{"),
        ('RBRACE', r"\}"),
        ('PONTO', r"\."),
        ("SEMI", r";"),
        ('NUM', r"[0-9]+"),
        ('SKIP', r"[ \t\r]+"),
        ('ERROR', r".")
    ]

In [13]:
import re

regex = re.compile("|".join(f"(?P<{n}>{p})" for n,p in tokens),
                flags=re.UNICODE|re.IGNORECASE)

def analisador(texto):
    reconhecidos = []
    linha = 1
    for m in regex.finditer(texto):
        tipo = m.lastgroup
        lex  = m.group()

        # gestão de linhas/ignorar espaços/comentários
        if tipo == "NEWLINE":
            linha += 1
            continue
        if tipo in ("SKIP", "COMMENT"):
            continue

        reconhecidos.append((tipo, lex, linha, m.span()))
    return reconhecidos


In [14]:
exemplo = '''
# DBPedia: obras de Chuck Berry
select ?nome ?desc where {
  ?s a dbo:MusicalArtist .
  ?s foaf:name "Chuck Berry"@en .
  ?w dbo:artist ?s .
  ?w foaf:name ?nome .
  ?w dbo:abstract ?desc
} limit 1000
'''
for i in analisador(exemplo):
    print(i)


('SELECT', 'select', 3, (33, 39))
('VAR', '?nome', 3, (40, 45))
('VAR', '?desc', 3, (46, 51))
('WHERE', 'where', 3, (52, 57))
('LBRACE', '{', 3, (58, 59))
('VAR', '?s', 4, (62, 64))
('A', 'a', 4, (65, 66))
('PREFIX', 'dbo:MusicalArtist', 4, (67, 84))
('PONTO', '.', 4, (85, 86))
('VAR', '?s', 5, (89, 91))
('PREFIX', 'foaf:name', 5, (92, 101))
('STRING', '"Chuck Berry"', 5, (102, 115))
('IDIOMA', '@en', 5, (115, 118))
('PONTO', '.', 5, (119, 120))
('VAR', '?w', 6, (123, 125))
('PREFIX', 'dbo:artist', 6, (126, 136))
('VAR', '?s', 6, (137, 139))
('PONTO', '.', 6, (140, 141))
('VAR', '?w', 7, (144, 146))
('PREFIX', 'foaf:name', 7, (147, 156))
('VAR', '?nome', 7, (157, 162))
('PONTO', '.', 7, (163, 164))
('VAR', '?w', 8, (167, 169))
('PREFIX', 'dbo:abstract', 8, (170, 182))
('VAR', '?desc', 8, (183, 188))
('RBRACE', '}', 9, (189, 190))
('LIMIT', 'limit', 9, (191, 196))
('NUM', '1000', 9, (197, 201))
